<a href="https://colab.research.google.com/github/shreyasguha/Shreyas_AI/blob/main/numpy_digit_classifier/MNIST_digitclassifier_2layer_10node.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple MNIST NN using numpy (2 layers, 10 neurons)

Here I implement a simple neural network with 1 input layer, 2 hidden layers, and 1 output layer for classifying the MNIST handwritten digit recognition database. Both the hidden layers contain 10 neurons, just like the output layer.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

In [5]:
pd_data = pd.read_csv('/content/drive/MyDrive/AI/Public Dataset/MNIST digit database/mnist_train.csv')
#pd_data

In [7]:
data = np.array(pd_data)
np.random.shuffle(data)

In [ ]:
# you should see 7, 3, 8.........0, 0, 4
data

In [9]:
m, n = data.shape

data_dev = data[0:1000].T
# Y_dev has 1 row, and columns from 1 to 1000 each column containing the actual label of that datapoint
Y_dev = data_dev[0]
# X_dev has 785 rows, each corresponding to different pixel values, and 1000 columns, each for a different datapoint
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
# Y_train has 1 row, and columns from 1000 to 60000 each column containing the actual label of that datapoint
Y_train = data_train[0]
# X_train has 785 rows, each corresponding to different pixel values, and 59000 columns, each for a different datapoint
X_train = data_train[1:n]
X_train = X_train / 255.
#_,m_train = X_train.shape

In [10]:
X_train.shape

(784, 59000)

Our NN will have a simple two-layer architecture. Input layer $a^{[0]}$ will have 784 units corresponding to the 784 pixels in each 28x28 input image. A hidden layer $a^{[1]}$ will have 10 units with ReLU activation, and finally our output layer $a^{[2]}$ will have 10 units corresponding to the ten digit classes with softmax activation.

**Forward propagation**

$$Z^{[1]} = W^{[1]} X + b^{[1]}$$
$$A^{[1]} = f(Z^{[1]}))$$
$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$
$$A^{[2]} = g(Z^{[2]})$$
$$Z^{[3]} = W^{[3]} A^{[2]} + b^{[3]}$$
$$A^{[3]} = softmax(Z^{[3]})$$

**Backward propagation**

$$dZ^{[3]} = A^{[3]} - Y$$
$$dW^{[3]} = \frac{1}{m} dZ^{[3]} A^{[2]T}$$
$$db^{[3]} = \frac{1}{m} \Sigma {dZ^{[3]}}$$
$$dZ^{[2]} = W^{[3]T} dZ^{[3]} .* f^{[2]\prime} (z^{[2]})$$
$$dW^{[2]} = \frac{1}{m} dZ^{[2]} A^{[1]T}$$
$$db^{[2]} = \frac{1}{m} \Sigma {dZ^{[2]}}$$
$$dZ^{[1]} = W^{[2]T} dZ^{[2]} .* g^{[1]\prime} (z^{[1]})$$
$$dW^{[1]} = \frac{1}{m} dZ^{[1]} X^{T}$$
$$db^{[1]} = \frac{1}{m} \Sigma {dZ^{[1]}}$$

**Parameter updates**

$$W^{[3]} := W^{[3]} - \alpha dW^{[3]}$$
$$b^{[3]} := b^{[3]} - \alpha db^{[3]}$$
$$W^{[2]} := W^{[2]} - \alpha dW^{[2]}$$
$$b^{[2]} := b^{[2]} - \alpha db^{[2]}$$
$$W^{[1]} := W^{[1]} - \alpha dW^{[1]}$$
$$b^{[1]} := b^{[1]} - \alpha db^{[1]}$$

**Vars and shapes**

Forward prop

- $A^{[0]} = X$: 784 x 1
- $Z^{[1]} \sim A^{[1]}$: 10 x 1
- $W^{[1]}$: 10 x 784 (as $W^{[1]} A^{[0]} \sim Z^{[1]}$)
- $B^{[1]}$: 10 x 1
- $Z^{[2]} \sim A^{[2]}$: 10 x 1
- $W^{[2]}$: 10 x 10 (as $W^{[2]} A^{[1]} \sim Z^{[2]}$)
- $B^{[2]}$: 10 x 1
- $W^{[3]}$: 10 x 10 (as $W^{[3]} A^{[2]} \sim Z^{[3]}$)
- $B^{[3]}$: 10 x 1


In [11]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    W3 = np.random.rand(10, 10) - 0.5
    b3 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2, W3, b3


In [12]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    W3 = np.random.rand(10, 10) - 0.5
    b3 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

def Sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def Sigmoid_deriv(Z):
    return np.exp(-Z) / pow(1 + np.exp(-Z), 2)

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y


#------------------------------------------MAKE CHANGES HERE FOR DIFFERENT ACTIVATION FUNCTIONS IN DIFFERENT LAYERS---------------------------------
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = Sigmoid(Z1)                      #here
    Z2 = W2.dot(A1) + b2
    A2 = ReLU(Z2)                         #here
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3)
    dZ2 = W3.T.dot(dZ3) * Sigmoid_deriv(Z2)  #here
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)  #here
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2, dW3, db3
#---------------------------------------------------------------------------------------------------------------------------------------------------

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3
    return W1, b1, W2, b2, W3, b3

In [13]:
def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent_start(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 50 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3

def gradient_descent_modify(W1, b1, W2, b2, W3, b3, X, Y, alpha, iterations):
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 50 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A3)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3


def make_predictions(X, W1, b1, W2, b2, W3, b3):
    _, _, _, _, _, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
    predictions = get_predictions(A3)
    return predictions

def test_prediction(index, W1, b1, W2, b2, W3, b3):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2, W3, b3)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)

    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
#W1, b1, W2, b2, W3, b3 = gradient_descent_start(X_train, Y_train, 0.10, 100)
#W1, b1, W2, b2, W3, b3 = init_params()
W1, b1, W2, b2, W3, b3 = gradient_descent_modify(W1, b1, W2, b2, W3, b3, X_train, Y_train, 0.1, 10)

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2, W3, b3)
get_accuracy(dev_predictions, Y_dev)

ReLU + ReLU
- On 500 timesteps we get 82.3% accuracy on our training data and 83.5% accuracy on our test data
- 500 more timesteps, train:87.4% test:87.6%
- 340 more timesteps to reach train:88.7% test:88.1%
- 1500 more timesteps, train:91.1% test:90%
- 2000 more timesteps, train:92.3% test:91.8%
-- at this point, at learning rate of 1, the accuracy seems to keep fluctuating around that 92.5% value, thus we reduce the learning rate to 0.5 and train further
- 2000 more timesteps, train:93.1% test:92.5%
- 2000 more timesteps, train:93.3% test:92.3%
